In [1]:
import pandas as pd #Analysis 
import matplotlib.pyplot as plt #Visulization
import seaborn as sns #Visulization
import numpy as np #Analysis 
from scipy.stats import norm #Analysis 
from sklearn.preprocessing import StandardScaler #Analysis 
from scipy import stats #Analysis 
import warnings 
warnings.filterwarnings('ignore')
%matplotlib inline
import gc

import os
import string
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn import model_selection, preprocessing, metrics, ensemble, naive_bayes, linear_model
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
import lightgbm as lgb

pd.options.mode.chained_assignment = None

## Importing Data

In [2]:
df_train = pd.read_csv(r"C:\Users\p_amb\Desktop\CapstoneFinal\DrugRec\www\Shuen's Data\kuc-hackathon-winter-2018\drugsComTrain_raw.csv", parse_dates=["date"])
df_test = pd.read_csv(r"C:\Users\p_amb\Desktop\CapstoneFinal\DrugRec\www\Shuen's Data\kuc-hackathon-winter-2018\drugsComTest_raw.csv", parse_dates=["date"])

In [3]:
df_all = pd.concat([df_train,df_test])

In [43]:
df_disease = pd.read_csv(r"C:\Users\p_amb\Desktop\CapstoneFinal\DrugRec\www\Shuen's Data\dia_t_clean.csv", encoding='latin-1')
df_symptoms = pd.read_csv(r"C:\Users\p_amb\Desktop\CapstoneFinal\DrugRec\www\Shuen's Data\sdsort\sym_t.csv")
df_dis_sym = pd.read_csv(r"C:\Users\p_amb\Desktop\CapstoneFinal\DrugRec\www\Shuen's Data\sdsort\diffsydiw.csv")

In [44]:
df_dis_sym = df_dis_sym[['did', 'syd', 'wei']]
df_dis_sym = df_dis_sym.sort_values(axis=0, by=['did'])
df_dis_sym = df_dis_sym.reset_index(drop=True)
df_dis_sym = df_dis_sym.rename(columns={"did": "disease", "syd": "symptom", "wei": "weight"})
df_dis_sym = df_dis_sym.drop(axis=0, index=5568)
df_dis_sym.head()

,disease,symptom,weight
0,1.0,7.0,1.0
1,1.0,275.0,1.0
2,1.0,173.0,1.0
3,1.0,54.0,1.0
4,1.0,32.0,1.0


In [45]:
df_disease.head()

,did,diagnose,description
0,1,Abdominal Aortic Aneurysm,enlarged major blood vessel
1,2,Abdominal Swelling,NaN
2,3,Abdominal Trauma,NaN
3,4,Abrasions,scrapes
4,5,ACE Inhibitor Induced Cough,blood pressure medication side effect/Drug-ind...


In [46]:
yea = list(df_disease.iloc[:,0])
len(yea)

1166

## Defining function that returns match percent of disease given symptoms

In [47]:
disease_list = []
for item in df_dis_sym.iloc[:, 0]:
    if item not in disease_list:
        disease_list.append(item)
len(disease_list)

1166

In [48]:
#creating new dataframe, condensed version of df_dis_sym
df_dis_sym_c = pd.DataFrame(columns={'disease', 'symptoms'})
sym_lst = []
for row in range(len(df_dis_sym)):
    if row == 0:
        sym_lst.append(df_dis_sym.iloc[row, 1])
    elif row == len(df_dis_sym)-1:
        sym_lst = sorted(sym_lst)
        df_dis_sym_c = df_dis_sym_c.append({'disease': df_dis_sym.iloc[row, 0], 'symptoms': sym_lst}, ignore_index=True)
    elif (df_dis_sym.iloc[row, 0] == df_dis_sym.iloc[row-1, 0]):
        sym_lst.append(df_dis_sym.iloc[row, 1])
    elif (df_dis_sym.iloc[row, 0] != df_dis_sym.iloc[row-1, 0]):
        sym_lst = sorted(sym_lst)
        df_dis_sym_c = df_dis_sym_c.append({'disease': df_dis_sym.iloc[row-1, 0], 'symptoms': sym_lst}, ignore_index=True)
        sym_lst = [df_dis_sym.iloc[row, 1]]
columnsTitles=["disease","symptoms"]
df_dis_sym_c=df_dis_sym_c.reindex(columns=columnsTitles)
df_dis_sym_c.head()
    

,disease,symptoms
0,1.0,"[7.0, 32.0, 54.0, 173.0, 205.0, 275.0]"
1,2.0,"[99.0, 284.0]"
2,3.0,"[2.0, 102.0]"
3,4.0,"[6.0, 11.0, 13.0, 23.0, 39.0, 41.0, 57.0, 67.0..."
4,5.0,"[17.0, 126.0, 159.0, 194.0, 195.0]"


In [49]:
# df_symptoms.head()
df_disease.head()

,did,diagnose,description
0,1,Abdominal Aortic Aneurysm,enlarged major blood vessel
1,2,Abdominal Swelling,NaN
2,3,Abdominal Trauma,NaN
3,4,Abrasions,scrapes
4,5,ACE Inhibitor Induced Cough,blood pressure medication side effect/Drug-ind...


In [50]:
#creating a dictionary of values with the keys being the syd indexes and the values being the symptom (index to symptom)
symp_dict = {}
for i in range(len(df_symptoms)):
    symp_dict[df_symptoms.iloc[i, 0]] = df_symptoms.iloc[i, 1]

In [51]:
#creating opposite dictionary (symptom to index)
symp_dict2 = {}
for i in range(len(df_symptoms)):
    symp_dict2[df_symptoms.iloc[i, 1]] = df_symptoms.iloc[i, 0]

In [52]:
#creating a dictionary of values with the keys being the did indexes and the values being the disease (index to disease)
dis_dict = {}
for i in range(len(df_disease)):
    dis_dict[df_disease.iloc[i, 0]] = df_disease.iloc[i, 1]

In [53]:
#creating opposite dictionary (disease to index)
dis_dict2 = {}
for i in range(len(df_disease)):
    dis_dict2[df_disease.iloc[i, 1]] = df_disease.iloc[i, 0]

In [54]:
#returns the top five diseases with matching symptoms, if ties then returns all
def return_disease(*symptoms):
    top_five = []
    value_lst = []
    per_match_dic = {}
    for row in range(len(df_dis_sym_c)):
        match = 0
        total = 0
        for symptom in symptoms:
            lst = df_dis_sym_c.iloc[row, 1]
            if symp_dict2[symptom] in lst:
                match += 1
                total += 1
            else:
                total += 1
        per_match = match/total
        disease_index = df_dis_sym_c.iloc[row, 0]
        disease = dis_dict[disease_index]
        per_match_dic[disease] = per_match
    for value in per_match_dic.values():
        value_lst.append(value)
    value_lst = sorted(value_lst)
    value_lst_top = value_lst[-1:-6:-1]
    for disease, value in per_match_dic.items():
        if value in value_lst_top:
            top_five.append((disease, value))
    return top_five

In [55]:
return_disease('Upper abdominal pain', 'Lower abdominal pain', 'Alcohol abuse')

[('Alcoholism', 0.6666666666666666),
 ('Appendicitis', 0.6666666666666666),
 ('Celiac Disease', 0.6666666666666666),
 ('Cholecystitis', 0.6666666666666666),
 ('Cholelithiasis', 0.6666666666666666),
 ('Colonic Diverticulitis', 0.6666666666666666),
 ('Constipation', 0.6666666666666666),
 ('Endometriosis', 0.6666666666666666),
 ('Gastric Ulcer', 0.6666666666666666),
 ('Gastroenteritis', 0.6666666666666666),
 ('Gastroesophageal Reflux', 0.6666666666666666),
 ('Hernia', 0.6666666666666666),
 ('Inguinal Hernia', 0.6666666666666666),
 ('Irritable Bowel Syndrome', 0.6666666666666666),
 ('Ligament Sprain', 0.6666666666666666),
 ('Kidney Stone', 0.6666666666666666),
 ('Ovarian Cyst', 0.6666666666666666),
 ('Pancreatitis', 0.6666666666666666),
 ('Pancreas Inflammation', 0.6666666666666666),
 ('Pyelonephritis', 0.6666666666666666),
 ('Bladder Infection', 0.6666666666666666),
 ('Umbilical Hernia', 0.6666666666666666),
 ('Lactose Intolerance', 0.6666666666666666),
 ('Diverticulosis', 0.6666666666666

In [56]:
#returns the top five diseases with matching symptoms, if ties then returns all, also return id
def return_disease2(*symptoms):
    top_five = []
    value_lst = []
    per_match_dic = {}
    for row in range(len(df_dis_sym_c)):
        match = 0
        total = 0
        for symptom in symptoms:
            lst = df_dis_sym_c.iloc[row, 1]
            if symp_dict2[symptom] in lst:
                match += 1
                total += 1
            else:
                total += 1
        per_match = match/total
        disease_index = df_dis_sym_c.iloc[row, 0]
        disease = dis_dict[disease_index]
        per_match_dic[(disease, disease_index)] = per_match
    for value in per_match_dic.values():
        value_lst.append(value)
    value_lst = sorted(value_lst)
    value_lst_top = value_lst[-1:-6:-1]
    for (disease, index), value in per_match_dic.items():
        if value in value_lst_top:
            top_five.append(((disease, index), value))
    return top_five

In [57]:
result = return_disease2('Upper abdominal pain', 'Lower abdominal pain', 'Alcohol abuse')

In [58]:
print(result)

[(('Alcoholism', 25.0), 0.6666666666666666), (('Appendicitis', 56.0), 0.6666666666666666), (('Celiac Disease', 139.0), 0.6666666666666666), (('Cholecystitis', 163.0), 0.6666666666666666), (('Cholelithiasis', 165.0), 0.6666666666666666), (('Colonic Diverticulitis', 179.0), 0.6666666666666666), (('Constipation', 187.0), 0.6666666666666666), (('Endometriosis', 241.0), 0.6666666666666666), (('Gastric Ulcer', 306.0), 0.6666666666666666), (('Gastroenteritis', 308.0), 0.6666666666666666), (('Gastroesophageal Reflux', 309.0), 0.6666666666666666), (('Hernia', 349.0), 0.6666666666666666), (('Inguinal Hernia', 388.0), 0.6666666666666666), (('Irritable Bowel Syndrome', 401.0), 0.6666666666666666), (('Ligament Sprain', 426.0), 0.6666666666666666), (('Kidney Stone', 513.0), 0.6666666666666666), (('Ovarian Cyst', 539.0), 0.6666666666666666), (('Pancreatitis', 546.0), 0.6666666666666666), (('Pancreas Inflammation', 547.0), 0.6666666666666666), (('Pyelonephritis', 637.0), 0.6666666666666666), (('Bladde

In [22]:
## Defining function that returns similiar condition in the new dataset

In [71]:
df = pd.read_csv(r"C:\Users\p_amb\Desktop\CapstoneFinal\DrugRec\www\Shuen's Data\kuc-hackathon-winter-2018\drugsComTrain_raw.csv")
df.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37


In [72]:

from collections import Counter 

counter = Counter(df['condition'])

conditions = counter.most_common()

print(conditions)



[('Birth Control', 28788), ('Depression', 9069), ('Pain', 6145), ('Anxiety', 5904), ('Acne', 5588), ('Bipolar Disorde', 4224), ('Insomnia', 3673), ('Weight Loss', 3609), ('Obesity', 3568), ('ADHD', 3383), ('Diabetes, Type 2', 2554), ('Emergency Contraception', 2463), ('High Blood Pressure', 2321), ('Vaginal Yeast Infection', 2274), ('Abnormal Uterine Bleeding', 2096), ('Bowel Preparation', 1859), ('ibromyalgia', 1791), ('Smoking Cessation', 1780), ('Migraine', 1694), ('Anxiety and Stress', 1663), ('Major Depressive Disorde', 1607), ('Constipation', 1595), ('Panic Disorde', 1463), ('Chronic Pain', 1455), ('Migraine Prevention', 1413), ('Urinary Tract Infection', 1316), ('Muscle Spasm', 1244), ('Osteoarthritis', 1239), ('Generalized Anxiety Disorde', 1164), ('Erectile Dysfunction', 1086), ('Opiate Dependence', 1079), ('Irritable Bowel Syndrome', 1014), ('Rheumatoid Arthritis', 995), ('Allergic Rhinitis', 973), ('Bacterial Infection', 945), ('Cough', 927), (nan, 899), ('Sinusitis', 835), 

In [73]:
conlist = []
reslist= []
for i in range(0,len(conditions)):
    conlist.append(conditions[i][0])

for i in range(0,len(result)):
    reslist.append(result[i][0][0])

print(reslist)

['Alcoholism', 'Appendicitis', 'Celiac Disease', 'Cholecystitis', 'Cholelithiasis', 'Colonic Diverticulitis', 'Constipation', 'Endometriosis', 'Gastric Ulcer', 'Gastroenteritis', 'Gastroesophageal Reflux', 'Hernia', 'Inguinal Hernia', 'Irritable Bowel Syndrome', 'Ligament Sprain', 'Kidney Stone', 'Ovarian Cyst', 'Pancreatitis', 'Pancreas Inflammation', 'Pyelonephritis', 'Bladder Infection', 'Umbilical Hernia', 'Lactose Intolerance', 'Diverticulosis', 'Ventral Hernia']


In [74]:
print(result)

[(('Alcoholism', 25.0), 0.6666666666666666), (('Appendicitis', 56.0), 0.6666666666666666), (('Celiac Disease', 139.0), 0.6666666666666666), (('Cholecystitis', 163.0), 0.6666666666666666), (('Cholelithiasis', 165.0), 0.6666666666666666), (('Colonic Diverticulitis', 179.0), 0.6666666666666666), (('Constipation', 187.0), 0.6666666666666666), (('Endometriosis', 241.0), 0.6666666666666666), (('Gastric Ulcer', 306.0), 0.6666666666666666), (('Gastroenteritis', 308.0), 0.6666666666666666), (('Gastroesophageal Reflux', 309.0), 0.6666666666666666), (('Hernia', 349.0), 0.6666666666666666), (('Inguinal Hernia', 388.0), 0.6666666666666666), (('Irritable Bowel Syndrome', 401.0), 0.6666666666666666), (('Ligament Sprain', 426.0), 0.6666666666666666), (('Kidney Stone', 513.0), 0.6666666666666666), (('Ovarian Cyst', 539.0), 0.6666666666666666), (('Pancreatitis', 546.0), 0.6666666666666666), (('Pancreas Inflammation', 547.0), 0.6666666666666666), (('Pyelonephritis', 637.0), 0.6666666666666666), (('Bladde

In [76]:
from fuzzywuzzy import fuzz



conlist = [i for i in conlist if type(i) is not float]


val = []
finallist = []
finallistindex  = []
for x in range(0,len(reslist)):
    val= []
    for y in range(0,len(conlist)):
        val.append(fuzz.token_sort_ratio(reslist[x],conlist[y]))
    values = np.array(val)
    searchval = max(val)
    ii = np.where(values == searchval)[0]
    for n in ii:
        finallist.append([conlist[n],val[n],x])
        
        
        
        
        
print(finallist)        

[['Alcohol Withdrawal', 57, 0], ['Sarcoidosis', 57, 0], ['Tendonitis', 64, 1], ['Meningitis', 64, 1], ["Behcet's Disease", 67, 2], ['Hepatitis C', 67, 3], ['Ichthyosis', 58, 4], ['Diverticulitis', 78, 5], ['Constipation', 100, 6], ['Endometriosis', 100, 7], ['Peptic Ulce', 67, 8], ['Gastroenteritis', 100, 9], ['Esophageal Spasm', 56, 10], ['Hypersomnia', 71, 11], ['Trigeminal Neuralgia', 63, 12], ['Irritable Bowel Syndrome', 100, 13], ['Light Sedation', 62, 14], ['Indigestion', 61, 15], ['Ovarian Cysts', 96, 16], ['Chronic Pancreatitis', 75, 17], ['Vaginal Yeast Infection', 59, 18], ['Postoperative Ocular Inflammation', 59, 18], ['Blepharitis', 72, 19], ['Bladder Infection', 100, 20], ['Head Lice', 56, 21], ['Local Anesthesia', 56, 21], ['Lactose Intolerance', 100, 22], ['Diverticulitis', 86, 23], ['Ventricular Tachycardia', 59, 24]]


In [77]:
from fuzzywuzzy import fuzz


def datatrsf(diseases, newconlist):
    reslist=[]
    for i in range(0,len(diseases)):
        reslist.append(diseases[i][0][0])
    
    
    
    val = []
    finallist = []
    finallistindex  = []
    for x in range(0,len(reslist)):
        val= []
        for y in range(0,len(newconlist)):
            val.append(fuzz.partial_ratio(reslist[x],newconlist[y]))
        values = np.array(val)
        searchval = max(val)
        ii = np.where(values == searchval)[0]
        for n in ii:
            finallist.append([newconlist[n],val[n],x])

    return finallist        

In [85]:
from collections import Counter 

counter = Counter(df['condition'])

conditions = counter.most_common()



conlist = []
for i in range(0,len(conditions)):
    conlist.append(conditions[i][0])


conlist = [i for i in conlist if type(i) is not float]

finaloutput = datatrsf(result, conlist)


print(finaloutput)


[['Alcohol Withdrawal', 80, 0], ['Meningitis', 74, 1], ['Inflammatory Bowel Disease', 80, 2], ['Post-Cholecystectomy Diarrhea', 77, 3], ['Cholera', 71, 4], ['Diverticulitis', 100, 5], ['Constipation', 100, 6], ['Constipation, Chronic', 100, 6], ['Constipation, Drug Induced', 100, 6], ['Chronic Idiopathic Constipation', 100, 6], ['Opioid-Induced Constipation', 100, 6], ['Constipation, Acute', 100, 6], ['Endometriosis', 100, 7], ['me', 100, 7], ['Gas', 100, 8], ['Gastric Ulcer Maintenance Treatment', 100, 8], ['Gastroenteritis', 100, 9], ['Gas', 100, 9], ['Campylobacter Gastroenteritis', 100, 9], ['Gas', 100, 10], ['Schizophrenia', 73, 11], ['Otitis Externa', 73, 11], ['Thrombocytopenia', 73, 11], ['Angina', 83, 12], ['Irritable Bowel Syndrome', 100, 13], ['me', 100, 13], ['me', 100, 14], ['Kidney Infections', 75, 15], ['Ovarian Cysts', 100, 16], ['Chronic Pancreatitis', 100, 17], ['Cance', 80, 18], ['Iritis', 83, 19], ['Bladder Infection', 100, 20], ['Hypersomnia', 71, 21], ['Lactose In

In [93]:
reslist

['Alcoholism',
 'Appendicitis',
 'Celiac Disease',
 'Cholecystitis',
 'Cholelithiasis',
 'Colonic Diverticulitis',
 'Constipation',
 'Endometriosis',
 'Gastric Ulcer',
 'Gastroenteritis',
 'Gastroesophageal Reflux',
 'Hernia',
 'Inguinal Hernia',
 'Irritable Bowel Syndrome',
 'Ligament Sprain',
 'Kidney Stone',
 'Ovarian Cyst',
 'Pancreatitis',
 'Pancreas Inflammation',
 'Pyelonephritis',
 'Bladder Infection',
 'Umbilical Hernia',
 'Lactose Intolerance',
 'Diverticulosis',
 'Ventral Hernia']

In [96]:

display = []
if(len(reslist)>5):
    for p in range(0,len(finaloutput)):
        constr = finaloutput[p][0]
        if(fuzz.ratio(constr , reslist[finaloutput[p][2]])>65):
            display.append(finaloutput[p])
            
if(len(display)<3):
    display = finaloutput
        
        

    


[['Diverticulitis', 100, 5],
 ['Constipation', 100, 6],
 ['Constipation, Chronic', 100, 6],
 ['Constipation, Acute', 100, 6],
 ['Endometriosis', 100, 7],
 ['Gastroenteritis', 100, 9],
 ['Campylobacter Gastroenteritis', 100, 9],
 ['Irritable Bowel Syndrome', 100, 13],
 ['Kidney Infections', 75, 15],
 ['Ovarian Cysts', 100, 16],
 ['Chronic Pancreatitis', 100, 17],
 ['Bladder Infection', 100, 20],
 ['Lactose Intolerance', 100, 22],
 ['Diverticulitis', 86, 23]]

In [97]:
result

[(('Alcoholism', 25.0), 0.6666666666666666),
 (('Appendicitis', 56.0), 0.6666666666666666),
 (('Celiac Disease', 139.0), 0.6666666666666666),
 (('Cholecystitis', 163.0), 0.6666666666666666),
 (('Cholelithiasis', 165.0), 0.6666666666666666),
 (('Colonic Diverticulitis', 179.0), 0.6666666666666666),
 (('Constipation', 187.0), 0.6666666666666666),
 (('Endometriosis', 241.0), 0.6666666666666666),
 (('Gastric Ulcer', 306.0), 0.6666666666666666),
 (('Gastroenteritis', 308.0), 0.6666666666666666),
 (('Gastroesophageal Reflux', 309.0), 0.6666666666666666),
 (('Hernia', 349.0), 0.6666666666666666),
 (('Inguinal Hernia', 388.0), 0.6666666666666666),
 (('Irritable Bowel Syndrome', 401.0), 0.6666666666666666),
 (('Ligament Sprain', 426.0), 0.6666666666666666),
 (('Kidney Stone', 513.0), 0.6666666666666666),
 (('Ovarian Cyst', 539.0), 0.6666666666666666),
 (('Pancreatitis', 546.0), 0.6666666666666666),
 (('Pancreas Inflammation', 547.0), 0.6666666666666666),
 (('Pyelonephritis', 637.0), 0.666666666

In [98]:
# druglist = conditions
# cleanedList = [i for i in druglist[0] if type(i) is str]
# print(fuzz.partial_ratio(result[0][0][0],cleanedList[36]))
for p in range(0,len(display)):
    
    display[p].append(result[display[2]][1])
    
    

TypeError: list indices must be integers or slices, not list

In [101]:
result = list(result)
display = list(display)
result[display[2]][1]



TypeError: list indices must be integers or slices, not list

In [127]:
df_disease['diagnose']
from collections import Counter 

Counter = Counter(df_disease['diagnose'])

commonDis = Counter.most_common()
commonDis1 =[]

commonDis[0][0]

for x in range(0,len(commonDis)):
    #print(commonDis[x][0])
    commonDis1.append(commonDis[x][0])

commonDis2=[]   
    
for x in commonDis1:
    y =  x.replace("\x0b"," ")
    z= y.replace("\x92 "," ")
    commonDis2.append(z)
    
print(commonDis2)

['Abdominal aortic aneurysm (enlarged major blood vessel)', 'Abdominal swelling', 'Abdominal trauma', 'Abrasions  (scrapes)', 'ACE inhibitor induced cough blood pressure medication side effect Drug-induced cough Drug reaction: ACE inhibitor', 'acetaminophen overdose Adverse reaction to acetaminophen', 'Tylenol  acetaminophen poisoning', 'Achilles tendonitis  (heel tendon inflammation)', 'Achilles tendon rupture (heel tendon tear)', 'Acid  LSD abuse', 'Acidosis  (excessive acid in the body)', 'Acoustic neuroma (ear nerve tumor)', 'AC joint separation acromioclavicular joint separation closed separation of acromioclavicular joint', 'Acute angle closure glaucoma increased inner eye pressure', 'Acute fatty liver of pregnancy', 'Adenoiditis (a type of lymph node inflammation)', 'Adenovirus infection  (virus infection)', 'Frozen shoulder  (adhesive capsulitis of shoulder)', 'Adjustment disorder  (poor adjustment to life stressor)', 'Alcohol  ethanol intoxication', 'Alcohol  ethanol abuse', '

In [118]:
commonDis1[0].replace("\x0b"," ")

'Abdominal aortic aneurysm (enlarged major blood vessel)'